# Amazon Keyword Insights — Analysis Notebook

In [ ]:
# --- Load & Clean the Amazon Cerebro dataset ---

import pandas as pd
import numpy as np
import glob, os

# Auto-pick a CSV inside data/
candidates = sorted(glob.glob(os.path.join("data", "*.csv")))
file_path = candidates[0] if candidates else r"data\YOUR_FILE.csv"

print("Using file:", file_path)

# Load
df = pd.read_csv(file_path)

# Basic cleanup
df = df.loc[:, ~df.columns.str.contains(r"^Unnamed", na=False)]
df.columns = df.columns.str.strip()

# Drop fully empty rows
df.dropna(how="all", inplace=True)

# Standardize column names to snake_case
def to_snake(s: str) -> str:
    s = s.strip().replace("%", "pct")
    s = s.replace("(", "").replace(")", "")
    s = s.replace(".", "").replace("/", "_")
    s = s.replace("-", " ")
    s = "_".join(s.split())
    return s.lower()

df.rename(columns={c: to_snake(c) for c in df.columns}, inplace=True)

# Coerce numeric where possible
for col in df.columns:
    if df[col].dtype == "object":
        coerced = pd.to_numeric(df[col].astype(str).str.replace(",", ""), errors="coerce")
        if coerced.notna().mean() >= 0.8:
            df[col] = coerced

# Handle missing values
num_cols = df.select_dtypes(include=[np.number]).columns
obj_cols = df.select_dtypes(exclude=[np.number]).columns
df[num_cols] = df[num_cols].apply(lambda s: s.fillna(s.median()))
df[obj_cols] = df[obj_cols].fillna("Unknown")

# Remove duplicates
df.drop_duplicates(inplace=True)

# Parse date-like columns
for col in df.columns:
    if "date" in col:
        df[col] = pd.to_datetime(df[col], errors="coerce")

df.reset_index(drop=True, inplace=True)

print("✅ Cleaned shape:", df.shape)
df.head()

In [ ]:
# --- Feature Engineering & Global Best/Top5 Keywords ---

import numpy as np
import pandas as pd

def _as_fraction(series: pd.Series) -> pd.Series:
    if series.empty:
        return series
    m = series.max(skipna=True)
    if pd.isna(m):
        return series
    if m <= 1.0:
        return series.clip(lower=0, upper=1)
    if m <= 100.0:
        return (series / 100.0).clip(lower=0, upper=1)
    return series

def safe_div(a, b):
    return np.where(b == 0, np.nan, a / b)

col_map = {
    "keyword": "keyword_phrase" if "keyword_phrase" in df.columns else None,
    "sv": "search_volume" if "search_volume" in df.columns else None,
    "sales": "keyword_sales" if "keyword_sales" in df.columns else None,
    "click_share": "aba_total_click_share" if "aba_total_click_share" in df.columns else None,
    "conv_share": "aba_total_conv_share" if "aba_total_conv_share" in df.columns else None,
    "sponsored_asins": "sponsored_asins" if "sponsored_asins" in df.columns else None,
    "competing_products": "competing_products" if "competing_products" in df.columns else None,
    "title_density": "title_density" if "title_density" in df.columns else None,
    "iq": "cerebro_iq_score" if "cerebro_iq_score" in df.columns else None,
}

df["ctr_est"] = _as_fraction(df[col_map["click_share"]]) if col_map["click_share"] else np.nan
df["cvr_est"] = _as_fraction(df[col_map["conv_share"]]) if col_map["conv_share"] else np.nan

if col_map["sv"]:
    df["est_clicks"] = df["ctr_est"] * df[col_map["sv"]]
    df["est_conversions"] = df["cvr_est"] * df["est_clicks"]
else:
    df["est_clicks"] = np.nan
    df["est_conversions"] = np.nan

if col_map["sv"] and col_map["sales"]:
    df["sales_per_search"] = safe_div(df[col_map["sales"]], df[col_map["sv"]])
else:
    df["sales_per_search"] = np.nan

if col_map["sv"] and col_map["competing_products"]:
    df["comp_per_1k_sv"] = safe_div(df[col_map["competing_products"]], (df[col_map["sv"]]/1000.0))
else:
    df["comp_per_1k_sv"] = np.nan

if col_map["sponsored_asins"]:
    max_sponsored = (df[col_map["sponsored_asins"]].max() or 1)
    df["sponsor_intensity"] = df[col_map["sponsored_asins"]] / max_sponsored
else:
    df["sponsor_intensity"] = np.nan

if col_map["sv"]:
    denom = (1.0 + df["comp_per_1k_sv"].fillna(df["comp_per_1k_sv"].median()))
    sponsor_term = (1.0 + df["sponsor_intensity"].fillna(df["sponsor_intensity"].median()))
    df["opportunity_index"] = safe_div(df[col_map["sv"]], denom * sponsor_term)
else:
    df["opportunity_index"] = np.nan

if col_map["title_density"]:
    df["opportunity_adj_title"] = safe_div(df["opportunity_index"], (1 + df[col_map["title_density"]]))
else:
    df["opportunity_adj_title"] = np.nan

if col_map["iq"] and col_map["sv"]:
    df["iq_per_1k_sv"] = safe_div(df[col_map["iq"]], (df[col_map["sv"]] / 1000.0))
else:
    df["iq_per_1k_sv"] = np.nan

# Save cleaned dataset
out_path = file_path.rsplit(".", 1)[0] + "_CLEANED.csv"
df.to_csv(out_path, index=False)
print("💾 Saved cleaned dataset to:", out_path)

# Global best/top5 keywords
kw_col = "keyword_phrase" if "keyword_phrase" in df.columns else None
sv_col = "search_volume" if "search_volume" in df.columns else None
sales_col = "keyword_sales" if "keyword_sales" in df.columns else None

if all(c in df.columns for c in [kw_col, sv_col, sales_col]):
    kw_df = df.dropna(subset=[sv_col, sales_col])
    kw_df = kw_df[kw_df[sv_col] > 0]
    kw_df["sales_per_search"] = kw_df[sales_col] / kw_df[sv_col]
    kw_df["score"] = kw_df[sv_col] * kw_df["sales_per_search"]
    best_keyword = kw_df.loc[kw_df["score"].idxmax(), [kw_col, sv_col, sales_col, "sales_per_search", "score"]]
    top5_keywords = kw_df.sort_values("score", ascending=False).head(5)
    print("✅ Best keyword overall:")
    display(best_keyword.to_frame().T)
    print("\n✅ Top 5 keywords overall:")
    display(top5_keywords)
else:
    print("Keyword-level columns not found; skipping global best/top5.")

In [ ]:
# --- ASIN-based mapping & plots ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

kw_col = "keyword_phrase" if "keyword_phrase" in df.columns else None
sv_col = "search_volume" if "search_volume" in df.columns else None
sales_col = "keyword_sales" if "keyword_sales" in df.columns else None

asin_cols = [c for c in df.columns if c.startswith("b0")]  # snake_cased ASIN columns

if kw_col and sv_col and sales_col and asin_cols:
    long_df = df.melt(
        id_vars=[kw_col, sv_col, sales_col],
        value_vars=asin_cols,
        var_name="asin",
        value_name="rank"
    )
    long_df = long_df.dropna(subset=["rank", sv_col, sales_col])
    long_df = long_df[long_df[sv_col] > 0]
    long_df["rank"] = pd.to_numeric(long_df["rank"], errors="coerce")
    long_df = long_df.dropna(subset=["rank"])

    long_df["sales_per_search"] = long_df[sales_col] / long_df[sv_col]
    long_df["rank_weight"] = 1 / (1 + long_df["rank"])
    long_df["score"] = long_df[sv_col] * long_df["sales_per_search"] * long_df["rank_weight"]

    best_per_asin = long_df.loc[
        long_df.groupby("asin")["score"].idxmax(),
        ["asin", kw_col, sv_col, sales_col, "sales_per_search", "rank", "score"]
    ].sort_values("score", ascending=False)

    print("✅ Best keyword per ASIN:")
    display(best_per_asin.head(20))

    top5_per_asin = (
        long_df.sort_values(["asin", "score"], ascending=[True, False])
               .groupby("asin")
               .head(5)
               .sort_values(["asin", "score"], ascending=[True, False])
               [[ "asin", kw_col, sv_col, sales_col, "sales_per_search", "rank", "score" ]]
    )
    print("\n✅ Top 5 keywords per ASIN:")
    display(top5_per_asin.head(50))

    # Save CSV outputs
    out_best = file_path.rsplit(".", 1)[0] + "_best_keyword_per_asin.csv"
    out_top5 = file_path.rsplit(".", 1)[0] + "_top5_keywords_per_asin.csv"
    best_per_asin.to_csv(out_best, index=False)
    top5_per_asin.to_csv(out_top5, index=False)
    print(f"\n💾 Saved results to:\n - {out_best}\n - {out_top5}")

    # Plots
    charts_dir = "charts"
    os.makedirs(charts_dir, exist_ok=True)

    # Use top5_keywords from previous cell if present
    if "top5_keywords" in globals():
        plt.figure(figsize=(10,6))
        plt.barh(top5_keywords[kw_col][::-1], top5_keywords["score"][::-1])
        plt.title("Top 5 Global Keywords by Score")
        plt.xlabel("Score")
        plt.ylabel("Keyword")
        plt.tight_layout()
        plt.savefig(os.path.join(charts_dir, "top5_global_keywords.png"), dpi=150)
        plt.show()

    # Plot for each ASIN (top 5), limit first 10 to avoid too many figures
    for asin in top5_per_asin["asin"].unique()[:10]:
        subset = top5_per_asin[top5_per_asin["asin"] == asin].copy()
        plt.figure(figsize=(10,6))
        plt.barh(subset[kw_col][::-1], subset["score"][::-1])
        plt.title(f"Top 5 Keywords for ASIN {asin.upper()}")
        plt.xlabel("Score")
        plt.ylabel("Keyword")
        plt.tight_layout()
        plt.savefig(os.path.join(charts_dir, f"top5_{asin}.png"), dpi=150)
        plt.show()
else:
    print("ASIN-based mapping skipped (required columns not found).")